In [20]:
# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from faces import detect_faces
import numpy as np
import argparse
import imutils
import time
import cv2
import pickle
import os

print("Imports Successful!!")


Imports Successful!!


In [21]:
DETECT_CONF_THRESH = 0.5
RECOG_CONF_THRESH = 0.5
detector_main_folder = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/LBP_face_recog/face_detector/"
recognizer_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/LBP_face_recog/output/caltech_full_recognizer_exp.yml"
label_encoder_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/LBP_face_recog/output/caltech_full_label_encoder_exp.pickle"


In [22]:
# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
prototxtPath = os.path.sep.join([detector_main_folder, "deploy.prototxt"])
weightsPath = os.path.sep.join([detector_main_folder, "res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNet(prototxtPath, weightsPath)

recognizer_path = recognizer_save_path
label_encoder_path = label_encoder_save_path
print("[INFO] loading trained face recognizer model...")
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read(recognizer_path)
le = pickle.loads(open(label_encoder_path, "rb").read())

# Get input feed
feed = 0
cam = cv2.VideoCapture(feed)
fps = []
confs = []
num_frames = 1
correct = 0
while True:
    ret, frame = cam.read()
    time_1 = time.time()
    boxes = detect_faces(net, frame)
#     print(boxes, time.time())
    image = frame.copy()
    # loop over the bounding boxes
    faces = []
    for (startX, startY, endX, endY) in boxes:
        # extract the face ROI, resize it, and convert it to
        # grayscale
        faceROI = image[startY:endY, startX:endX].copy()
        faceROI = cv2.resize(faceROI, (47, 62))
        faceROI = cv2.cvtColor(faceROI, cv2.COLOR_BGR2GRAY)
        faces.append(faceROI)
        
    for face in faces:
        (prediction, conf) = recognizer.predict(face)
        predName = le.inverse_transform([prediction])[0]
        if predName == "shubhranshu_malhotra":
            correct+=1
        text = "{}: {:.2f}%".format(predName, conf)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(image, (startX, startY), (endX, endY),
                        (0, 0, 255), 2)
        cv2.putText(image, text, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,0), 2)
        confs.append(conf)
    

    cv2.imshow("detector", image)
    time_2 = time.time()
    fps.append(1/(time_2 - time_1))
    if num_frames == 100:        
        print(f"Accuracy: {correct}")
        print(f"Average confidence: {np.mean(confs):.2f}")
        print(f"Average FPS: {np.mean(fps):.2f}")
        correct = 0
        confs = []
        fps = []
        num_frames = 0
    num_frames += 1
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()



[INFO] loading face detector model...
[INFO] loading trained face recognizer model...
Accuracy: 84
Average confidence: 87.27
Average FPS: 16.42
Accuracy: 88
Average confidence: 87.01
Average FPS: 16.69
Accuracy: 88
Average confidence: 87.41
Average FPS: 16.13
Accuracy: 86
Average confidence: 87.67
Average FPS: 15.42
Accuracy: 84
Average confidence: 87.13
Average FPS: 15.81
Accuracy: 58
Average confidence: 79.31
Average FPS: 16.90
Accuracy: 70
Average confidence: 85.22
Average FPS: 17.11
Accuracy: 85
Average confidence: 85.96
Average FPS: 16.33
Accuracy: 69
Average confidence: 85.86
Average FPS: 15.95
Accuracy: 87
Average confidence: 88.97
Average FPS: 15.61
Accuracy: 87
Average confidence: 84.77
Average FPS: 16.34
Accuracy: 59
Average confidence: 85.71
Average FPS: 17.28
Accuracy: 67
Average confidence: 88.84
Average FPS: 17.30
Accuracy: 42
Average confidence: 88.26
Average FPS: 18.74
Accuracy: 62
Average confidence: 87.31
Average FPS: 16.49
Accuracy: 31
Average confidence: 85.68
Avera

### recognizer
`Near:`  
* Average FPS: 16.95  
* Average confidence: 74.76  
* Accuracy: 99

`Mid:`  
* Accuracy: 83
* Average FPS: 16
* Average confidence: 80

`Mid_Far:`  
* Accuracy: 35
* Average FPS: 16.43
* Average confidence: 76.92

**accuracy decreases with distance. Allover lower accuracy than 2**  

### recognizer_2:
`Near:` 
* Accuracy: 99
* Average confidence: 70.61
* Average FPS: 15.50

`Mid:`  
* Average confidence: 74.30
* Average FPS: 15.39
* Accuracy: 89

`Mid_Far:`  
* Accuracy: 66
* Average confidence: 74.64
* Average FPS: 15.44

**consistent accuracy, good accuracy, 

## Experiments

### [radius=1, neighbors=8, grid_x=6, grid_y=6]

`Near:`  
* Accuracy: 99
* Average confidence: 64.15
* Average FPS: 15.96

`Mid:`  
* Accuracy: 93
* Average confidence: 66.64
* Average FPS: 15.94

`Mid_Far:`  
* Accuracy: 75
* Average confidence: 66.23
* Average FPS: 16.09




### [radius=3, neighbors=8, grid_x=6, grid_y=6]

`Near:`  
* Accuracy: 85
* Average confidence: 87.27
* Average FPS: 16.42

`Mid:`  
* Accuracy: 77
* Average confidence: 85.96
* Average FPS: 16.33

`Mid_Far:`  
* Accuracy: 62
* Average confidence: 87.31
* Average FPS: 16.49



### [radius=10, neighbors=8, grid_x=6, grid_y=6]

`Near:`  
* Accuracy: 70
* Average confidence: 97.14
* Average FPS: 18.37

`Mid:`  
* Accuracy: 56
* Average confidence: 97.92
* Average FPS: 17.53

`Mid_Far:`  
* Accuracy: 25
* Average confidence: 101.57
* Average FPS: 18.10


### [radius=1, neighbors=8, grid_x=6, grid_y=6]

`Near:`  

`Mid:`  

`Mid_Far:`  

### [radius=5, neighbors=8, grid_x=6, grid_y=6]

`Near:`  

`Mid:`  

`Mid_Far:`  

### [radius=7, neighbors=8, grid_x=6, grid_y=6]

`Near:`  

`Mid:`  

`Mid_Far:`  
